In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
cd /content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Code

/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Code


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math
from scipy import signal
from PIL import Image
import argparse

# Copy the path of the folder that contains this notebook using the file navigation on the left:
# Ex. /content/drive/My\ Drive/CIS 581-Online/Canny Edge Project/Learner Code and Images/Code
sys.path.append('/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_template/Code')

# import functions
from helpers import interp2, edgeLink

ModuleNotFoundError: ignored

# Tests and Visualization

In [ ]:
def Test_script(I, E):
    test_pass = True

    # E should be 2D matrix
    if E.ndim != 2:
      print('ERROR: Incorrect Edge map dimension! \n')
      print(E.ndim)
      test_pass = False
    # end if

    # E should have same size with original image
    nr_I, nc_I = I.shape[0], I.shape[1]
    nr_E, nc_E = E.shape[0], E.shape[1]

    if nr_I != nr_E or nc_I != nc_E:
      print('ERROR: Edge map size has changed during operations! \n')
      test_pass = False
    # end if

    # E should be a binary matrix so that element should be either 1 or 0
    numEle = E.size
    numOnes, numZeros = E[E == 1].size, E[E == 0].size

    if numEle != (numOnes + numZeros):
      print('ERROR: Edge map is not binary one! \n')
      test_pass = False
    # end if

    if test_pass:
      print('Shape Test Passed! \n')
    else:
      print('Shape Test Failed! \n')

    return test_pass

In [ ]:
'''
  Derivatives visualzation function
'''
def visDerivatives(I_gray, Mag, Magx, Magy):
    fig, (Ax0, Ax1, Ax2, Ax3) = plt.subplots(1, 4, figsize = (20, 8))

    Ax0.imshow(Mag, cmap='gray', interpolation='nearest')
    Ax0.axis('off')
    Ax0.set_title('Gradient Magnitude')

    Ax1.imshow(Magx, cmap='gray', interpolation='nearest')
    Ax1.axis('off')
    Ax1.set_title('Gradient Magnitude (x axis)')
    
    Ax2.imshow(Magy, cmap='gray', interpolation='nearest')
    Ax2.axis('off')
    Ax2.set_title('Gradient Magnitude (y axis)')

    # plot gradient orientation
    Mag_vec = Mag.transpose().reshape(1, Mag.shape[0] * Mag.shape[1]) 
    hist, bin_edge = np.histogram(Mag_vec.transpose(), 100)

    ind_array = np.array(np.where( (np.cumsum(hist).astype(float) / hist.sum()) < 0.95))
    thr = bin_edge[ind_array[0, -1]]

    ind_remove = np.where(np.abs(Mag) < thr)
    Magx[ind_remove] = 0
    Magy[ind_remove] = 0

    X, Y = np.meshgrid(np.arange(0, Mag.shape[1], 1), np.arange(0, Mag.shape[0], 1))

    ori = Ax3.imshow(I_gray, cmap='hsv')
    Ax3.axis('off')
    Ax3.set_title('Gradient Orientation')
    fig.colorbar(ori, ax=Ax3, )
    
    Q = plt.quiver(X, Y, Magx, Magy)
    qk = plt.quiverkey(Q, 0.9, 0.9, 2, r'$2 \frac{m}{s}$', labelpos='E', coordinates='figure')


'''
  Edge detection result visualization function
'''
def visCannyEdge(Im_raw, M, E):
    # plot image
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize = (12, 12))

    # plot original image
    ax0.imshow(Im_raw)
    ax0.axis("off")
    ax0.set_title('Raw image')

    # plot edge detection result
    ax1.imshow(M, cmap='gray', interpolation='nearest')
    ax1.axis("off")
    ax1.set_title('Non-Max Suppression Result')

    # plot original image
    ax2.imshow(E, cmap='gray', interpolation='nearest')
    ax2.axis("off") 
    ax2.set_title('Canny Edge Detection')

# Functions

In [ ]:
'''
  Convert RGB image to gray one manually
  - Input I_rgb: 3-dimensional rgb image
  - Output I_gray: 2-dimensional grayscale image
'''
def rgb2gray(I_rgb):
    r, g, b = I_rgb[:, :, 0], I_rgb[:, :, 1], I_rgb[:, :, 2]
    I_gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return I_gray

In [ ]:
def findDerivatives(I_gray):
    '''
    File clarification:
        Compute gradient information of the input grayscale image
        - Input I_gray: H x W matrix as image
        - Output Mag: H x W matrix represents the magnitude of derivatives
        - Output Magx: H x W matrix represents the magnitude of derivatives along x-axis
        - Output Magy: H x W matrix represents the magnitude of derivatives along y-axis
        - Output Ori: H x W matrix represents the orientation of derivatives
    '''
    # Set Gaussian
    G = np.array([[2, 4, 5, 4, 2], [4, 9, 12, 9, 4], [5, 12, 15, 12, 5], [4, 9, 12, 9, 4], [2, 4, 5, 3, 2]])
    G = (1/159) * G
    G = np.reshape(G,(5,5))
    
    # Set dx gradient
    dx1 = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    dx1 = np.reshape(dx1,(3,3))
    dx1 = -dx1

    # Set dy gradient
    dy1 = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]])
    dy1 = np.reshape(dy1,(3,3))

    # Convolution of image with Gaussian
    Gx = signal.convolve2d(G, dx1, 'same')
    Gy = signal.convolve2d(G, dy1, 'same')

    # Convolution of image with Gx and Gy
    Magx = signal.convolve2d(I_gray, Gx, 'same')
    Magy = signal.convolve2d(I_gray, Gy, 'same')

    # Calculate Mag of xy and the edge orientation
    Mag = np.sqrt(Magy**2 + Magx**2)
    Ori = np.arctan2(Magy, Magx)

    return Mag, Magx, Magy, Ori

NameError: ignored

In [ ]:
def nonMaxSup(Mag, Ori):
    '''
    File clarification:
        Find local maximum edge pixel using NMS along the line of the gradient
        - Input Mag: H x W matrix represents the magnitude of derivatives
        - Input Ori: H x W matrix represents the orientation of derivatives
        - Output M: H x W binary matrix represents the edge map after non-maximum suppression
    '''
    nr, nc = Mag.shape
    M = np.zeros(Mag.shape)
    x, y = np.meshgrid(np.arange(nc), np.arange(nr))
    # getting neighbor in the oritention direction
    neighbor1_x = np.clip(x + np.cos(Ori), 0, nc - 1)
    neighbor1_y = np.clip(y + np.sin(Ori), 0, nr - 1)
    # using interpolation to get neighbor
    neighbor1 = interp2(Mag, neighbor1_x, neighbor1_y)
    # getting neighbor in the opposite of the oritention direction
    # TODO: get neighbor 2
    neighbor2_x = np.clip(x - np.cos(Ori), 0, nc - 1)
    neighbor2_y = np.clip(y - np.sin(Ori), 0, nr - 1)
    neighbor2 = interp2(Mag, neighbor2_x, neighbor2_y)
    # TODO: perform NMS
    neighbor1_1d = neighbor1.flatten()
    neighbor2_1d = neighbor2.flatten()
    Mag = Mag.flatten()
    M = M.flatten()
    for i in range(Mag.size):
      if Mag[i] >= neighbor1_1d[i] and Mag[i] >= neighbor2_1d[i]:
        M[i] = 1
    M = M.reshape(nr,nc)
    return M


In [ ]:
def cannyEdge(I, low, high):
    # convert RGB image to gray color space
    im_gray = rgb2gray(I)

    Mag, Magx, Magy, Ori = findDerivatives(im_gray)
    M = nonMaxSup(Mag, Ori)
    E = edgeLink(M, Mag, Ori, low, high)

    # only when test passed that can show all results
    if Test_script(im_gray, E):
        # visualization results
        visDerivatives(im_gray, Mag, Magx, Magy)
        visCannyEdge(I, M, E)

        plt.show()

    return E

Tune the threshold for each images 

In [ ]:
 # list all image names
 os.listdir('/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Images')

NameError: ignored

In [ ]:
image_folder = "/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Images"
save_folder = "/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Results" # need to create this folder in the drive
image_name = 'hq_test.jpg'
filename = image_name # TODO: change image name 
I = np.array(Image.open(os.path.join(image_folder, filename)).convert('RGB'))
low, high = 20, 50

E = cannyEdge(I, low, high)
pil_image = Image.fromarray(E.astype(np.uint8) * 255).convert('L')
pil_image.save(os.path.join(save_folder, "{}_Result.png".format(filename.split(".")[0])))

NameError: ignored

# Fill in all tuned threshold to generate edge detection results


In [ ]:
# Copy the path of the folder that contains the images using the file navigation on the left:
# Ex. /content/drive/My\ Drive/CIS 581-Online/Canny Edge Project/Learner Code and Images/Images
image_folder = "/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Images"
save_folder = "/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Projects/Project 2 Canny Edge Detection/Canny_Edge_learner_hanyangl/Results"
# fill in the threshold (low, high) you have tuned in the cell above 
thresh_dict = {'118035.jpg': (3.2, 7),
                '135069.jpg': (3.1, 7.8),
                '16068.jpg': (20, 35),
                '189080.jpg': (20, 30),
                '201080.jpg': (10, 35),
                '21077.jpg': (15, 40),
                '22013.jpg': (20, 50),
                '3096.jpg': (15, 30),
                '48017.jpg': (6, 15),
                '55067.jpg': (0.4, 14),
                '86000.jpg': (20, 30),
                'I1.jpg': (10, 20)}
# generate results one by one
for filename in os.listdir(image_folder):
    # read in image 
    im_path = os.path.join(image_folder, filename)
    I = np.array(Image.open(im_path).convert('RGB'))

    low, high = thresh_dict[filename]
    E = cannyEdge(I, low, high)

    pil_image = Image.fromarray(E.astype(np.uint8) * 255).convert('L')

    pil_image.save(os.path.join(save_folder, "{}_Result.png".format(filename.split(".")[0])))